# Tutorial to access MUR SST on AWS from your local computer  

This will run on your local computer, but will be slower than running it on AWS.


Credits: Tutorial development
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org)    - Farallon Institute, USA
* [Dr. Rich Signell](mailto:rsignell@usgs.gov) - USGS

Credits: Creating of the Zarr MUR SST dataset
* [Aimee Barciauskas](mailto:aimee@developmentseed.org) - Development Seed
* [Dr. Rich Signell](mailto:rsignell@usgs.gov) - USGS
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org)    - Farallon Institute, USA
* [Joseph Flasher](mailto:jflasher@amazon.com) - AWS
-------------

## Please note that this is global, 1 km, daily data.  This is a very large dataset and the analyses below can take up to 5-10 minutes depending on your internet speed.  A better option is to run your notebook on AWS or Google Cloud where it will be much faster.


# Structure of this tutorial

1. Opening data
2. Data exploration
2. Data plotting

# 1. Opening data

-------------------

## Import python packages

It is nice to turn off warnings and set xarray display options

In [ ]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import fsspec

warnings.simplefilter('ignore') # filter some warning messages
xr.set_options(display_style="html")  #display dataset nicely 

### Initialize Dataset

### Read MUR SST from AWS public dataset program on an s3 bucket.  This Pangeo binder is running on Google Cloud and data access takes longer because the data is on AWS.  

This code is an example of how to read from a s3 bucket.  Right now (2/14/2020) this takes ~1min on AWS and ~2 min on google cloud, in part because of how we chunked the data in time.  We should have the data re-chunked in time by 3/1/2020 and then it should read in less than 30 sec.

In [ ]:
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent
Printing the dataset will show you the dimensions, coordinates, and data variables with clickable icons at the end that show more metadata and size.

In [ ]:
ds_sst

# 2.  Explore the data

#### Let's explore the data

- With all data, it is important to explore it and understand what is contains before doing an analysis.
- The ice mask used by MUR SST is from NSIDC and is based on satellite passive microwave estimates of sea ice concentration
- The satellite data isn't available near land, so the is no estimate of sea ice concentration near land
- For this data, it means that there are some erroneous SSTs near land, that is likely ice and this is something to be aware of.

In [ ]:
sst = ds_sst['analysed_sst']

cond = (ds_sst.mask==1) & ((ds_sst.sea_ice_fraction<.15) | np.isnan(ds_sst.sea_ice_fraction))
sst_masked = ds_sst['analysed_sst'].where(cond)

# 3. Data plotting

``xarray`` plotting functions rely on matplotlib internally, but they make use of all available metadata to make the plotting operations more intuitive and interpretable. More plotting examples are given [here](http://xarray.pydata.org/en/stable/plotting.html)

### Plot SST on 10/1/2015

Select the ``analysed_sst`` variable on a specific day and load the data into memory.

In [ ]:
%%time
sst_day = sst_masked.sel(time='2015-10-01',lat=slice(20,65),lon=slice(-170,-110)).load()

#### Check how the masked data looks

In [ ]:
sst_day.plot()

# xarray can do more!

* concatentaion
* open network located files with openDAP
* import and export Pandas DataFrames
* .nc dump to 
* groupby_bins
* resampling and reduction

For more details, read this blog post: http://continuum.io/blog/xray-dask


## Where can I find more info?

### For more information about xarray

- Read the [online documentation](http://xarray.pydata.org/)
- Ask questions on [StackOverflow](http://stackoverflow.com/questions/tagged/python-xarray)
- View the source code and file bug reports on [GitHub](http://github.com/pydata/xarray/)

### For more doing data analysis with Python:

- Thomas Wiecki, [A modern guide to getting started with Data Science and Python](http://twiecki.github.io/blog/2014/11/18/python-for-data-science/)
- Wes McKinney, [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) (book)

### Packages building on xarray for the geophysical sciences


- [eofs](https://github.com/ajdawson/eofs): empirical orthogonal functions by Andrew Dawson
- [infinite-diff](https://github.com/spencerahill/infinite-diff) by Spencer Hill 
- [aospy](https://github.com/spencerahill/aospy) by Spencer Hill and Spencer Clark
- [regionmask](https://github.com/mathause/regionmask) by Mathias Hauser
- [salem](https://github.com/fmaussion/salem) by Fabien Maussion

Resources for teaching and learning xarray in geosciences:
- [Fabien's teaching repo](https://github.com/fmaussion/teaching): courses that combine teaching climatology and xarray
